# 载入模型并预测测试集，输出结果

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import time

In [2]:
test_feature = pd.read_csv("data/test_feature.csv")
test_data = pd.read_csv("data/A_testData0531.csv")

In [3]:
def load_model(name):
    model = joblib.load("saved_model/" + name)
    return model

def get_timeStamp(timeString, form=0):
    if form == 0:
        timeArray = time.strptime(timeString, '%Y-%m-%dT%H:%M:%S.%fZ')
    if form == 1:
        timeArray = time.strptime(timeString, '%Y/%m/%d  %H:%M:%S')
    timeStamp = int(time.mktime(timeArray))
    return timeStamp

def timestamp_to_str(timestamp=None, format='%Y/%m/%d  %H:%M:%S'):
    if timestamp:
        time_tuple = time.localtime(timestamp)  # 把时间戳转换成时间元祖
        result = time.strftime(format, time_tuple)  # 把时间元祖转换成格式化好的时间
        return result
    else:
        return time.strptime(format)
    
def model_test(model, test_feature, test_data):
    _test_feature = test_feature.copy()
    _test_data = test_data.copy()
    test_feature_array = np.array(_test_feature.iloc[:,1:])
    _test_feature['label'] = model.predict(test_feature_array)
    result = _test_feature[['loadingOrder','label']]

    _test_data = _test_data.merge(result, on='loadingOrder', how='left')
    _test_data['ETA'] = _test_data['onboardDate'].apply(lambda x:get_timeStamp(x,form=1)) + _test_data['label']
    _test_data['ETA'] = _test_data['ETA'].apply(lambda x:timestamp_to_str(x))
    _test_data.drop(['direction','TRANSPORT_TRACE'],axis=1,inplace=True)
    #test_data['onboardDate'] = test_data['onboardDate']
    _test_data['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')
    # 整理columns顺序
    result = _test_data[['loadingOrder', 'timestamp', 'longitude', 'latitude', 'carrierName', 'vesselMMSI', 'onboardDate', 'ETA', 'creatDate']]
    return result

In [4]:
model_xgb = load_model("xlg.model")
result = model_test(model_xgb, test_feature, test_data)

In [5]:
result.to_csv('result/result_xgb.csv', index=False)